# TileDB Backend for xarray (Partially Filled Arrays)

## About this Example

### What it shows

This example shows some of the basic usage for opening a TileDB array in xarray using the TileDB backend when the TileDB array is not fully filled.

### Set-up Requirements
This example requires `tiledb-cf` to be installed and uses the `tiledb`, `xarray`, and `numpy` libraries. 

In [ ]:
import tiledb
import xarray as xr
import numpy as np

In [ ]:
# Set names for the output generated by the example.
output_dir = "output/tiledb-xarray-partially-filled"
uri1 = f"{output_dir}/example1"
uri2 = f"{output_dir}/example2"

In [ ]:
# Reset output folder
import os
import shutil

shutil.rmtree(output_dir, ignore_errors=True)
os.mkdir(output_dir)

## Example 1: Simple partially-filled 2D array

In [ ]:
# Create array and write data.
tiledb.Array.create(
    uri1,
    tiledb.ArraySchema(
        domain=tiledb.Domain(
            tiledb.Dim("x", domain=(0, 7), dtype=np.uint64),
            tiledb.Dim("y", domain=(0, 7), dtype=np.uint64),
        ),
        attrs=[tiledb.Attr("z", np.float64)],
    ),
)
with tiledb.open(uri1, mode="w") as array:
    array[2:6, 2:6] = np.reshape(np.arange(16), (4, 4))

In [ ]:
# Print non-empty domain and data.
with tiledb.open(uri1) as array:
    print(f"Non-empty domain: {array.nonempty_domain()}")
    print(f"Data in non-empty domain:\n {array.multi_index[:, :]['z']}")
    print(f"All data: \n {array[:, :]['z']}")

In [ ]:
# By default, xarray will only open the non-empty domain
dataset = xr.open_dataset(uri1, engine="tiledb")
dataset

In [ ]:
# To open the full dataset set `open_full_domain=True`
full_dataset = xr.open_dataset(uri1, open_full_domain=True, engine="tiledb")
full_dataset

## Example 2: Array with a datetime dimensions

In [ ]:
# Create array and write data.
tiledb.Array.create(
    uri2,
    tiledb.ArraySchema(
        domain=tiledb.Domain(
            tiledb.Dim(
                "date",
                domain=(np.datetime64("2000", "D"), np.datetime64("2000-12-31", "D")),
                dtype=np.dtype("datetime64[D]"),
            ),
        ),
        attrs=[tiledb.Attr("z", np.float64)],
    ),
)
with tiledb.open(uri2, mode="w") as array:
    array[np.datetime64("2000-03", "D") : np.datetime64("2000-03-31", "D")] = np.arange(
        1, 32
    )

In [ ]:
# Print non-empty domain and data.
with tiledb.open(uri2) as array:
    print(f"Non-empty domain: {array.nonempty_domain()}")
    print(f"Data in non-empty domain:\n {array.multi_index[:]['z']}")

In [ ]:
# Open data in xarray
date_dataset = xr.open_dataset(uri2, engine="tiledb")
date_dataset